# Importing Needed Packages

In [19]:
import pandas as pd
import numpy as np
from math import sqrt

# Reading the data 
#### we have 2 data Frame which we have to analyze the ratings and each user

In [20]:
mdf= pd.read_csv('F:\DATA\MAIN\/movieDF.csv')
rdf= pd.read_csv('F:\DATA\MAIN/ratingDF.csv')

mdf.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


 ### Separating the year from the name with regex and then add a new column named year and add the years to the new column

In [21]:
mdf['year'] = mdf.title.str.extract('(\(\d\d\d\d\))',expand=False)
mdf['year'] = mdf.year.str.extract('(\d\d\d\d)',expand=False)
mdf['title'] = mdf['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()

# turning genres to array (replacing | to , )
mdf['genres']=mdf.genres.str.split('|')
mdf.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


# seperating each genre
## by this lines we can seperate each genre and add it to a new column

In [22]:
moviesWithGenre = mdf.copy()
for index , row in mdf.iterrows():
    for genre in row['genres']:
            moviesWithGenre.at[index , genre]=1
moviesWithGenre=moviesWithGenre.fillna(0).drop('genres' , axis=1)
moviesWithGenre.head()


,movieId,title,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### here we will delete the timestamp column because in this case it is not important 

In [23]:
rdf=rdf.drop('timestamp' , axis=1)
rdf.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


# Adding a new User 
# Quality assessment with a random user


In [24]:

userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating': 3.5},
            {'title':'Jumanji', 'rating': 2.5},
            {'title':"Pulp Fiction", 'rating': 5},
            {'title':'Akira', 'rating': 4},
            {'title':'TMNT (Teenage Mutant Ninja Turtles)' , 'rating':4}
         ] 

inputMovies=pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.5
3,Pulp Fiction,5.0
4,Akira,4.0
5,TMNT (Teenage Mutant Ninja Turtles),4.0


# User Genres
#### in this part we seperate the genres which the user likes and add them into a new table

In [25]:
userGenreTable=moviesWithGenre[moviesWithGenre['title'].isin(inputMovies['title'])].drop('movieId', axis=1).drop('title', axis=1).drop('year', axis=1)
userGenreTable=userGenreTable.reset_index(drop=True)
userGenreTable.transpose()
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# it's time to multiple 
### by taking an average we can give a score to each genre . then we know the rate of each genre for the new user

In [26]:
userProfile=userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

Adventure             17.5
Animation             14.0
Children              12.5
Comedy                15.5
Fantasy               12.5
Romance                0.0
Drama                  6.5
Action                 9.0
Crime                  2.5
Thriller               2.5
Horror                 0.0
Mystery                0.0
Sci-Fi                 5.0
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

# genre table for all movies
### here we have to add 1( has ) and 0 ( has not ) for every genres for each movie to multiple in user's table 

In [27]:
genreTable=moviesWithGenre.set_index(moviesWithGenre['movieId']).drop('title' , axis=1).drop('movieId' , axis=1).drop('year' , axis=1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Soretd Reconneder Table
### By multiplying the user's table to the moive's genres we undrestand which movies does the new user like

In [28]:
recommender_table=((genreTable*userProfile).sum(axis=1)/userProfile.sum()).sort_values(ascending=False)
recommender_table

movieId
26340     0.830769
108932    0.830769
51939     0.830769
134853    0.805128
32031     0.789744
            ...   
1329      0.000000
1328      0.000000
1326      0.000000
1325      0.000000
3280      0.000000
Length: 9742, dtype: float64

# filtering top 20 movies of the table 
### Then we sort them and show.

In [29]:
mdf.loc[mdf['movieId'].isin(recommender_table.head(20).keys())]

,movieId,title,genres,year
478,546,Super Mario Bros.,"[Action, Adventure, Children, Comedy, Fantasy,...",1993
559,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
3568,4886,"Monsters, Inc.","[Adventure, Animation, Children, Comedy, Fantasy]",2001
5490,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
5819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
5977,36397,Valiant,"[Adventure, Animation, Children, Comedy, Fanta...",2005
6047,40339,Chicken Little,"[Action, Adventure, Animation, Children, Comed...",2005
6194,45074,"Wild, The","[Adventure, Animation, Children, Comedy, Fantasy]",2006
6448,51939,TMNT (Teenage Mutant Ninja Turtles),"[Action, Adventure, Animation, Children, Comed...",2007


## With scores, we can analyze and see what kind of movies users like, and then by finding movies that have genres that users like, we can recommend the top 20 movies they should watch.
#### - First, we separate the user's favorite genres with a score (interest rate) and then by multiplying the user's matrix in all movie genres, which are 1 and 0, and get the average, we have a special code for each movie, which is the percentage of similarity, and we show it to the user by sorting them.


# Now let's do this with Collabrative Filtering 

#### Merging user input with movies database to get movieId

In [30]:
inputMovies = inputMovies.merge(mdf[['movieId', 'title']], on='title', how='left')
inputMovies

,title,rating,movieId
0,"Breakfast Club, The",5.0,1968
1,Toy Story,3.5,1
2,Jumanji,2.5,2
3,Pulp Fiction,5.0,296
4,Akira,4.0,1274
5,TMNT (Teenage Mutant Ninja Turtles),4.0,51939


### Converting movieId to integer

In [31]:
inputMovies['movieId'] = inputMovies['movieId'].astype(int)
inputMovies

,title,rating,movieId
0,"Breakfast Club, The",5.0,1968
1,Toy Story,3.5,1
2,Jumanji,2.5,2
3,Pulp Fiction,5.0,296
4,Akira,4.0,1274
5,TMNT (Teenage Mutant Ninja Turtles),4.0,51939


#### Filtering ratings of users who have rated videos similar to the new user

In [32]:
userSubset = rdf[rdf['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset

,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0
...,...,...,...
99534,610,1,5.0
99552,610,296,5.0
99636,610,1274,5.0
99664,610,1968,4.0


### Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter


In [33]:

userSubsetGroup = userSubset.groupby('userId')
userSubsetGroup.get_group(103)

,userId,movieId,rating
15565,103,1,4.0
15566,103,2,4.0
15587,103,296,5.0
15684,103,1968,4.0


#### Sorting it so users with movie most in common with the input will have priority


In [34]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

[(177,
         userId  movieId  rating
  24900     177        1     5.0
  24901     177        2     3.5
  24930     177      296     5.0
  25069     177     1274     2.0
  25129     177     1968     3.5
  25569     177    51939     2.5),
 (477,
         userId  movieId  rating
  75424     477        1     4.0
  75425     477        2     4.0
  75447     477      296     4.5
  75558     477     1274     5.0
  75613     477     1968     4.0
  75950     477    51939     4.0),
 (91,
         userId  movieId  rating
  14121      91        1     4.0
  14122      91        2     3.0
  14173      91      296     4.5
  14316      91     1274     5.0
  14383      91     1968     3.0)]

### Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient


In [35]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
    if isinstance(name, tuple):
        name = name[0] 
    name = int(name) 
    
    group = group.sort_values(by='movieId')
    inputMoviesSorted = inputMovies.sort_values(by='movieId')
    
    nRatings = len(group)
    
    temp_df = inputMoviesSorted[inputMoviesSorted['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList), 2) / float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList), 2) / float(nRatings)
    Sxy = sum(i * j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList) * sum(tempGroupList) / float(nRatings)
    
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy / sqrt(Sxx * Syy)
    else:
        pearsonCorrelationDict[name] = 0

#### Converting dictionary to dataframe and convert userId data type to int64

In [39]:
similarity_df = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
similarity_df.columns = ['similarityIndex']
similarity_df['userId'] = similarity_df.index
similarity_df['userId'] = similarity_df['userId'].astype('int64')  # تبدیل نوع داده به int64
similarity_df.index = range(len(similarity_df))
similarity_df.head()

,similarityIndex,userId
0,0.127343,177
1,0.251976,477
2,0.263523,91
3,0.542326,219
4,0.753080,274


#### Get similar users (highest correlation) by sorting similarity_df from max similarity to min

In [41]:
topUsers = similarity_df.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
83,1.0,489
45,1.0,132
65,1.0,305
101,1.0,5
125,1.0,112


# Get points from similar users 
### Weighting scores based on correlation
#### at last we have to Calculate the weighted average for each movie


In [44]:
topUsersRating = topUsers.merge(rdf, left_on='userId', right_on='userId', how='inner')
topUsersRating['weightedRating'] = topUsersRating['similarityIndex'] * topUsersRating['rating']
topUsersRatingGroup = topUsersRating.groupby('movieId').agg({'similarityIndex': 'sum', 'weightedRating': 'sum'})
topUsersRatingGroup['weightedAverageRecommendationScore'] = topUsersRatingGroup['weightedRating'] / topUsersRatingGroup['similarityIndex']
topUsersRatingGroup = topUsersRatingGroup.sort_values(by='weightedAverageRecommendationScore', ascending=False)
topUsersRatingGroup.head()

,similarityIndex,weightedRating,weightedAverageRecommendationScore
movieId,,,
140265,1.0,5.0,5.0
1956,1.0,5.0,5.0
69529,1.0,5.0,5.0
151769,1.0,5.0,5.0
69275,1.0,5.0,5.0


### Get moviess that the new user hasn't seen yet

#### then we have to Integrate with movie database to get movie information and Filter movies with a high recommendation score (more than 3.5)


In [46]:
recommendedMovies = topUsersRatingGroup[~topUsersRatingGroup.index.isin(inputMovies['movieId'])]
recommendedMovies = recommendedMovies.merge(mdf[['movieId', 'title', 'genres', 'year']], left_index=True, right_on='movieId')
recommendedMovies = recommendedMovies[recommendedMovies['weightedAverageRecommendationScore'] > 3.5]
recommendedMovies.head()

,similarityIndex,weightedRating,weightedAverageRecommendationScore,movieId,title,genres,year
9012,1.0,5.0,5.0,140265,George Carlin: Jammin' in New York,[Comedy],1992
1433,1.0,5.0,5.0,1956,Ordinary People,[Drama],1980
7065,1.0,5.0,5.0,69529,Home,[Documentary],2009
9214,1.0,5.0,5.0,151769,Three from Prostokvashino,[Animation],1978
7051,1.0,5.0,5.0,69275,Dead Snow (Død snø),"[Action, Adventure, Comedy, Horror]",2009


##  Sort and select top 20 movies

In [47]:
recommendedMovies = recommendedMovies[['movieId', 'title', 'genres', 'year', 'weightedAverageRecommendationScore']].head(20)
recommendedMovies

,movieId,title,genres,year,weightedAverageRecommendationScore
9012,140265,George Carlin: Jammin' in New York,[Comedy],1992,5.0
1433,1956,Ordinary People,[Drama],1980,5.0
7065,69529,Home,[Documentary],2009,5.0
9214,151769,Three from Prostokvashino,[Animation],1978,5.0
7051,69275,Dead Snow (Død snø),"[Action, Adventure, Comedy, Horror]",2009,5.0
1703,2290,Stardust Memories,"[Comedy, Drama]",1980,5.0
932,1232,Stalker,"[Drama, Mystery, Sci-Fi]",1979,5.0
2707,3632,Monsieur Verdoux,"[Comedy, Crime]",1947,5.0
7003,67788,Confessions of a Shopaholic,"[Comedy, Romance]",2009,5.0
384,441,Dazed and Confused,[Comedy],1993,5.0


### Based on these two methods and comparing the results, we realize that these methods are different, but we can find similar movies in each recommendation table.
 